# Librerias para el web scrapping

In [1]:
!pip install requests
!pip install bs4
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

# Importamos las librerias que vamos a necesitar Para el Web Scrapping

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import bs4

## Importamos url base

In [3]:
baseurl ='https://statsroyale.com/es'

Conseguimos un agente de usuario que pueda enviar con cada solicitud, por lo qque de 
forma predeterminada cuando el usuario solicita si envia a una pagina el agente de 
usuario es la solicitud de Python y creo que eso puede obtener mas bloques a menudo, así
que en realidad vamos a anular y eligí este aqui desde este sitio web es común que se envíe y diga que estamos en windows y estamso usando MOZILLA 


In [4]:
headers ={'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 10_0 like Mac OS X) AppleWebKit/602.4.6 (KHTML, like Gecko) Version/10.0 Mobile/14A346 Safari/E7FBAF'}

Ponemos la URL  de la web de donde comenzaremos a sacar la informacion 

In [5]:
r = requests.get('https://statsroyale.com/es/cards')

usamos beatifulsoup para obtener todo de la pagina y voy a usar el analizador XML earle en 

In [6]:
soup = BeautifulSoup(r.content, 'lxml')

Extraemos todos los link que encontramos que se relacionan directamente con la informacion que queremos extraer

In [7]:
cardlist =soup.find_all('div',class_="cards__card")

guardamos los links

In [8]:
cardlinks =[]
for item in cardlist:
    for link in item.find_all('a', href= True): 
        cardlinks.append(link['href'])

Verificamos la cantidad de link que extraemos 

In [9]:
print('Numero de links:',len(cardlinks))

Numero de links: 102


En esta parte se comienza a extraer la informaciond de cada carta 

In [10]:
lista_diccionario = []

In [11]:
!pip install regex

#### Patrones para clasificar las cartas.

In [12]:
import re
#Patrones 
      #GOLEN Y GIGANTES
patron_gigante = re.compile(r'\b([G|g]\w+te)|([G|g][o|ó]\w+m)\b')
      #muerto
patron_muerto = re.compile(r'(\b[E|e]sq\w+[o|s])|([R|L][o|á]\w+[s|a])\b')
      #CRIATURAS O CARTAS HIELO
patron_criatura =re.compile(r'\b(([H|h]i\w+o)|([n|F][i|a]\w+[a|e]))\b')
      #CARTAS DRAGON
patron_dragon = re.compile(r'\b([D|d]r\w+\W*[n|s])\b')
      #CARTAS DE DUENDES
patron_duende = re.compile(r'\b([D|d]u\w+[e|s])\b')
      #CARTAS DE MAGOS Y BRUJAS
patron_magico = re.compile(r'\b((Ma\w+o)|(Br\w+a))\b')
      #CARTAS DE ESTRUCTURAS (CHOZAS)
patron_choza = re.compile(r'\b((Ch\w+a))\b')
      # CARTAS DE ESTRUCTURAS TORRES
patron_torre = re.compile(r'\b((To\w+e))\b')
      #CARTAS GUERREROS HUMANOS
patron_humano = re.compile(r'\b((P[r|e]\W*\w+[a|e|r])|(Ca\w+[r|o])|([M|g|B][a|u|e|i]\w+[e|i][d|r][o|a])|(([V|M|m]\w+as*)))\b')

### Lista de atributos que se extraeran:
    Nombre                             name
    Nivel                              level
    Calidad                            quality
    Objetivo                           objective
    Alcance                            scope
    Velocidad                          speed
    Radio                              radio
    Tiempo de vida                     time of life
    Puntos de vida                     Life points
    Daño                               Hurt
    Daño\s                             Damage
    Daño de área                       Area damage

In [13]:
for link in cardlinks:
    r = requests.get(link, headers = headers)
    soup = BeautifulSoup(r.content,'lxml')
    #####################################################
    #Creamos variables de aquella informacion que no tiene una clase ni etiqueta que los distinga 
    # ademas que puede o no que existan estas variable en cada uno de los links
    lifepoints = 'None'
    hurt = 'None'
    damage = 'None'
    areadamage = 'None'
    leve = soup.find('tbody')
    
    try:
        #recorremos todos los valores de etiqueta con la clase CARD_TABLEVALUES
        for item in leve.find_all('tr',class_='card__tableValues'):
            if 'Puntos de vida'== item.find('strong').text.strip():
                lifepoints = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño'== item.find('strong').text.strip():
                hurt = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño/s'== item.find('strong').text.strip():
                damage = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño de área' == item.find('strong').text.strip():
                areadamage = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            #print(type(item.find('strong').text.strip()))
            #print(item.find('strong').text.strip(), item.find('td',class_ ='card__tableValue card__isActive').text.strip())
            #print(item.find('td',class_ ='card__tableValue card__isActive').text)
    except:
        lifepoints = 'None'
        hurt = 'None'
        damage = 'None'
        areadamage = 'None'
####################################################################

    name =soup.find('div',class_='ui__headerMedium card__cardName').text.strip()
    #############################################################################################
    classes = []
    if patron_gigante.search(name):
        classes.append('gigante')
    if patron_muerto.search(name):
        classes.append('muerto')
    if patron_criatura.search(name):
        classes.append('criatura')
    if patron_dragon.search(name):
        classes.append('dragon')
    if patron_duende.search(name):
        classes.append('duende')
    if patron_magico.search(name):
        classes.append('magico')
    if patron_choza.search(name):
        classes.append('choza')
    if patron_torre.search(name):
        classes.append('torre')
    if patron_humano.search(name):
        classes.append('humano')
    else:
        classes.append('otro')
    #############################################################################################
    try:
        level = soup.find('th', class_='card__tableHeader card__isActiveHeader').text.strip()
    except:
        level = 'None'
    try:
        quality = soup.find('span',class_='card__rare').text.strip()
    except:
        try:
            quality = soup.find('span',class_='card__legendary').text.strip()
        except:
            try:
                quality = soup.find('span',class_='card__epic').text.strip()
            except:
                try:
                    quality = soup.find('span',class_='card__common').text.strip()
                except:
                    quality = 'None'
    try:
        objective = soup.find('div', class_='card__metric card__target').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        objective = 'None'
    try:
        scope = soup.find('div',class_='card__metric card__range').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        scope = 'None'
    try:
        speed = soup.find('div',class_ ='card__metric card__movementSpeed').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        speed = 'None'
    try:
        radio = soup.find('div',class_='card__metric card__radius').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        radio = 'None'
    try:
        timeoflife = soup.find('div',class_ = 'card__metric card__durationLifetime').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        timeoflife = 'None'
#Creamos la estructura del diccionario
    CARDS ={
        'name':name,
        'level':level,
        'quality':quality,
        'objective': objective,
        'scope':scope,
        'speed':speed,
        'radio': radio,
        'timeoflife':timeoflife,
        ######################################
        'lifepoints' : lifepoints,
        'hurt' : hurt,
        'damage' : damage,
        'areadamage' : areadamage,
        'link':link,
        'classes': classes
    }
    #guardamos los diccionarios en una lista
    lista_diccionario.append(CARDS)

Verificamos la cantidad de diccionarios en la lista

In [14]:
# Número de Cartas
print(len(lista_diccionario))

102


In [15]:
for i in range(len(lista_diccionario)):
    print(lista_diccionario[i]['name'],":",lista_diccionario[i]['classes'])

Trío de mosqueteras : ['humano']
Gólem : ['gigante', 'otro']
Gigante eléctrico : ['gigante', 'otro']
Reclutas reales : ['otro']
P.E.K.K.A. : ['otro']
Sabueso de lava : ['otro']
Megacaballero : ['humano']
Gigante noble : ['gigante', 'otro']
Bárbaros de élite : ['otro']
Esqueleto gigante : ['gigante', 'muerto', 'otro']
Duende gigante : ['gigante', 'duende', 'otro']
Chispitas : ['otro']
Bárbaros : ['otro']
Horda de esbirros : ['otro']
Pillos : ['otro']
Globo bombástico : ['otro']
Bruja : ['magico', 'otro']
Príncipe : ['humano']
Lanzarrocas : ['otro']
Verdugo : ['otro']
Cañón con ruedas : ['otro']
Dragón eléctrico : ['dragon', 'otro']
Montacarneros : ['otro']
Gigante : ['gigante', 'otro']
Puercos reales : ['otro']
Mago : ['magico', 'otro']
Dragones esqueleto : ['muerto', 'dragon', 'otro']
Bebé dragón : ['dragon', 'otro']
Príncipe oscuro : ['humano']
Cazador : ['humano']
Leñador : ['otro']
Dragón infernal : ['dragon', 'otro']
Mago eléctrico : ['magico', 'otro']
Bruja nocturna : ['magico', '

# Modelando Grafo

In [16]:
# install rdflib 
!pip3 install rdflib

In [17]:
# Nodes 
from rdflib import URIRef, BNode, Literal
# Namespace 
from rdflib import Namespace
# resources
from rdflib.namespace import RDF, RDFS
# Graph 
from rdflib import ConjunctiveGraph
import pprint

## Clash Royale
Clash Royale es el juego de lucha de ritmo rápido en el que recolectas cartas y luchas contra otros jugadores en tiempo real. Defiende tus torres y destruye las de tu enemigo para ganar la partida.
## Cartas
#### Características:
- Nombre de la carta             
- Nivel                           
- calidad de carta                       
- Objetivo                      
- Velocidad                                  
- Radio 
- Tiempo de vida                  
- Puntos de vida
- Daño
- Daño\s   
- Daño de área 

#### Tipo de Cartas:
1. Tropa
2. Estructura
3. Hechizo

# TEST
busqueda = 'name'
for dic in lista_diccionario[:71]:
    print(dic['name'],":",dic[busqueda])

# TEST
for dic in lista_diccionario[71:83]:
    print(dic['name'],":",dic[busqueda])

# TEST 
for dic in lista_diccionario[83:]:
    print(dic['name'],":",dic[busqueda])

# Clases Extra 
## tropa
    1. Duendes
    2. Gigantes
    3. dragones
    4. muertos
    5. criaturas
    6. magico
    7. humanos
## estructuras
    1. Chozas
    2. Torres
## Hechizos
    1. nada.

In [18]:
n = Namespace("https://statsroyale.com/es/")

In [19]:
# subpropertys of property characteristic
name = n.name
level = n.level
quality = n.quality
objective = n.objective
scope = n.scope
speed = n.speed
radio = n.radio
timeoflife = n.timeoflife
lifepoints = n.lifepoints
hurt = n.hurt
damage = n.damage
areadamage = n.areadamage

# subproperties Quality 
rare = n.rare
epic = n.epic
common = n.common
legendary = n.legendary

# subproperties objective
groundandair = n.groundandair
building = n.building
ground = n.ground

# subproperties scope
melee = n.melee
near = n.near
far = n.far

# subproperties speed
slow = n.slow
medium = n.medium
fast = n.fast
veryfast = n.veryfast



In [20]:
g= ConjunctiveGraph()
print(g)

[a rdflib:ConjunctiveGraph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]


In [21]:

# subclases de clase maestra: Objeto y subclases sucesoras
g.add((n.troop, RDFS.subClassOf, n.card))
g.add((n.build, RDFS.subClassOf, n.card))
g.add((n.spell, RDFS.subClassOf, n.card))

# subclases de la carta tropa
g.add([n.giant, RDFS.subClassOf, n.troop])
g.add([n.dead, RDFS.subClassOf, n.troop])
g.add([n.creature, RDFS.subClassOf, n.troop])
g.add([n.dragon, RDFS.subClassOf, n.troop])
g.add([n.goblin, RDFS.subClassOf, n.troop])
g.add([n.human, RDFS.subClassOf, n.troop])
g.add([n.magic, RDFS.subClassOf, n.troop])

# subclases de la carta estructura
g.add([n.hut, RDFS.subClassOf, n.build])
g.add([n.tower, RDFS.subClassOf, n.build])




In [22]:
# Subpropiedades de la clase Caracteristica
g.add([name, RDFS.subPropertyOf, n.characteristic])
g.add([level, RDFS.subPropertyOf, n.characteristic])
g.add([quality, RDFS.subPropertyOf, n.characteristic])
g.add([objective, RDFS.subPropertyOf, n.characteristic])
g.add([scope, RDFS.subPropertyOf, n.characteristic])
g.add([speed, RDFS.subPropertyOf, n.characteristic])
g.add([radio, RDFS.subPropertyOf, n.characteristic])
g.add([timeoflife, RDFS.subPropertyOf, n.characteristic])
g.add([lifepoints, RDFS.subPropertyOf, n.characteristic])
g.add([hurt, RDFS.subPropertyOf, n.characteristic])
g.add([damage, RDFS.subPropertyOf, n.characteristic])
g.add([areadamage, RDFS.subPropertyOf, n.characteristic])

# subpropiedades de la calidad
g.add([rare ,RDFS.subPropertyOf, quality ])
g.add([epic ,RDFS.subPropertyOf, quality ])
g.add([common ,RDFS.subPropertyOf, quality ])
g.add([legendary ,RDFS.subPropertyOf, quality ])

# subpropiedades de objetivo
g.add([groundandair , RDFS.subPropertyOf, objective ])
g.add([building , RDFS.subPropertyOf, objective ])
g.add([ground , RDFS.subPropertyOf, objective ])

#subpropiedades de alcance
g.add([melee ,RDFS.subPropertyOf, scope ])
g.add([near ,RDFS.subPropertyOf, scope ])
g.add([far ,RDFS.subPropertyOf, scope ])

#subpropiedades de velocidad
g.add([slow ,RDFS.subPropertyOf, speed ])
g.add([medium ,RDFS.subPropertyOf, speed ])
g.add([fast ,RDFS.subPropertyOf, speed ])
g.add([veryfast ,RDFS.subPropertyOf, speed ])

#propiedades dominios y rango
g.add([timeoflife, RDFS.domain, n.spell ])
g.add([timeoflife, RDFS.domain, n.build ])




In [23]:
# nombres de las clases principales
g.add([n.card, name , Literal('cartas')])
g.add([n.troop, name , Literal('tropas')])
g.add([n.build, name , Literal('estructuras')])
g.add([n.spell, name , Literal('hechizos')])
g.add([n.charasteristic, name, Literal('caracteristicas')])

# nombres de las subclases
g.add([n.giant, name, Literal('gigante')])
g.add([n.dead, name, Literal('muerto')])
g.add([n.creature, name, Literal('criatura')])
g.add([n.dragon, name, Literal('dragon')])
g.add([n.goblin, name, Literal('duende')])
g.add([n.magic, name, Literal('magico')])
g.add([n.human, name,Literal('human')])

g.add([n.hut, name,Literal('choza')])
g.add([n.tower, name,Literal('torre')])

# nombre de las calidades
g.add([rare , name, Literal('Especial')])
g.add([epic , name, Literal('Epico')])
g.add([common , name, Literal('común') ])
g.add([legendary , name, Literal('Legendaria') ])

#bnombres de los objetivos
g.add([groundandair , name, Literal('De tierra y aire') ])
g.add([building , name, Literal('Estructura') ])
g.add([ground , name, Literal('Ground') ])

# nombres de los alcances
g.add([melee , name, Literal('Cuerpo a cuerpo') ])
g.add([near , name, Literal('cerca') ])
g.add([far , name, Literal('lejos') ])

g.add([slow , name, Literal('Baja')])
g.add([medium , name, Literal('Media') ])
g.add([fast , name, Literal('Alta') ])
g.add([veryfast , name, Literal('Muy Alta') ])

# Funciones de transformación

In [24]:

def fun_quality(statement):
    if(statement == 'Especial'):
        return rare
    if(statement == 'Épica'):
        return epic
    if(statement == 'Común'):
        return common
    if(statement == 'Legendaria'):
        return legendary

In [25]:
def fun_objective(statement):
    if(statement == 'De tierra y aire'):
        return groundandair
    if(statement == 'Estructuras'):
        return building
    if(statement == 'Terrestres'):
        return ground

In [26]:
def fun_scope(statement):
    if(statement == 'Cuerpo a cuerpo'):
        return melee
    else:
        num = float(statement)
        if(num < 8.0):
            return near
        else:
            return far

In [27]:
def fun_speed(statement):
    if(statement=='Baja'):
        return slow
    if(statement=='Media'):
        return medium
    if(statement=='Alta'):
        return fast
    if(statement=='Muy alta'):
        return veryfast

In [28]:
# funcion que se le pasa por parametro el array de clases, el link y la clase general por si no lo ubica.
def fun_clase(arr_class,link,general_class):
    for clase in arr_class:
        if(clase == 'gigante'):
            g.add([ URIRef(link) ,RDF.type,n.giant])
        if(clase == 'muerto'):
            g.add([ URIRef(link) ,RDF.type,n.dead])
        if(clase == 'criatura'):
            g.add([ URIRef(link) ,RDF.type,n.creature])
        if(clase == 'dragon'):
            g.add([ URIRef(link) ,RDF.type,n.dragon])
        if(clase == 'duende'):
            g.add([ URIRef(link) ,RDF.type,n.goblin])
        if(clase == 'magico'):
            g.add([ URIRef(link) ,RDF.type,n.magic])
        if(clase == 'choza'):
            g.add([ URIRef(link) ,RDF.type,n.hut])
        if(clase == 'torre'):
            g.add([ URIRef(link) ,RDF.type,n.tower])
        if(clase == 'humano'):
            g.add([ URIRef(link) ,RDF.type,n.human])
        else:
            g.add([ URIRef(link) ,RDF.type,general_class])

# TEST
busqueda = 'speed'
i=0
for dic in lista_diccionario:
    i=i+1
    print(i)
    if(dic[busqueda]!= 'None'):
        print(dic['name'],":",fun_speed(dic[busqueda]))

In [29]:
# Instanciando Cartas Tropas

for diccionario in lista_diccionario[:71]:
    
    #g.add((URIRef(diccionario['link']), RDF.type, n.troop))
    
    fun_clase(diccionario['classes'],diccionario['link'],n.troop)
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        g.add((URIRef(diccionario['link']), objective, fun_objective(diccionario['objective'])))
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    #if (diccionario['timeoflife']!= 'None'):
        #g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['lifepoints']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))
    


In [30]:
# Instanciando Estructuras
for diccionario in lista_diccionario[71:83]:
    #g.add((URIRef(diccionario['link']), RDF.type, n.build))
    fun_clase(diccionario['classes'],diccionario['link'],n.build)
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        g.add((URIRef(diccionario['link']), objective, fun_objective(diccionario['objective'])))
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    if (diccionario['timeoflife']!= 'None'):
        g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))

In [31]:
# Instanciando Hechizos
for diccionario in lista_diccionario[83:]:
    #g.add((URIRef(diccionario['link']), RDF.type, n.spell))
    fun_clase(diccionario['classes'],diccionario['link'],n.spell)
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        g.add((URIRef(diccionario['link']), objective, fun_objective(diccionario['objective'])))
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    if (diccionario['timeoflife']!= 'None'):
        g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))

texto = str(g.serialize(format="n3", encoding='utf-8').decode())
for line in texto.splitlines():
    print(line)

In [32]:
# Exportar el archivo como :
g.serialize(destination='statsroyale.rdf', format='xml')

In [33]:
#imprimir tripletas
for s, p, o in g:
    print(s, p, o)

https://statsroyale.com/es/card/Princess https://statsroyale.com/es/areadamage 140
https://statsroyale.com/es/card/Elixir+Golem https://statsroyale.com/es/hurt 211
https://statsroyale.com/es/card/Electro+Wizard https://statsroyale.com/es/areadamage 75
https://statsroyale.com/es/card/Ice+Golem https://statsroyale.com/es/objective https://statsroyale.com/es/building
https://statsroyale.com/es/ground http://www.w3.org/2000/01/rdf-schema#subPropertyOf https://statsroyale.com/es/objective
https://statsroyale.com/es/duende https://statsroyale.com/es/name duende
https://statsroyale.com/es/card/Goblin+Giant https://statsroyale.com/es/speed https://statsroyale.com/es/medium
https://statsroyale.com/es/card/Lava+Hound https://statsroyale.com/es/name Sabueso de lava
https://statsroyale.com/es/card/Royal+Giant https://statsroyale.com/es/level 9
https://statsroyale.com/es/card/Giant https://statsroyale.com/es/damage 140
https://statsroyale.com/es/card/Tesla https://statsroyale.com/es/hurt 190
https:

https://statsroyale.com/es/card/Graveyard https://statsroyale.com/es/lifepoints 67
https://statsroyale.com/es/card/Guards https://statsroyale.com/es/speed https://statsroyale.com/es/fast
https://statsroyale.com/es/card/Royal+Giant https://statsroyale.com/es/hurt 254
https://statsroyale.com/es/card/Giant https://statsroyale.com/es/lifepoints 3275
https://statsroyale.com/es/card/Flying+Machine https://statsroyale.com/es/speed https://statsroyale.com/es/fast
https://statsroyale.com/es/card/Night+Witch https://statsroyale.com/es/speed https://statsroyale.com/es/medium
https://statsroyale.com/es/card/Lightning https://statsroyale.com/es/timeoflife 1.5s
https://statsroyale.com/es/card/Royal+Giant https://statsroyale.com/es/objective https://statsroyale.com/es/building
https://statsroyale.com/es/card/Bomb+Tower http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://statsroyale.com/es/tower
https://statsroyale.com/es/card/Witch https://statsroyale.com/es/quality https://statsroyale.com/es/epi

# TEST : Remover Tripletas
for s, p, o in g:
    g.remove((s, p, o))

# inferencias

#### Clases superiores

In [36]:
clase_base = n.giant
for s in g.transitive_objects(clase_base, RDFS.subClassOf):
    print(g.value(s, name))

gigante
tropas
cartas


##### Clases inferiores

In [37]:
clase_base = n.card
for s in g.transitive_subjects (RDFS.subClassOf,clase_base):
    print(s,g.value(s, name))

https://statsroyale.com/es/card cartas
https://statsroyale.com/es/build estructuras
https://statsroyale.com/es/tower torre
https://statsroyale.com/es/hut None
https://statsroyale.com/es/spell hechizos
https://statsroyale.com/es/troop tropas
https://statsroyale.com/es/dead muerto
https://statsroyale.com/es/creature criatura
https://statsroyale.com/es/human human
https://statsroyale.com/es/magic magico
https://statsroyale.com/es/duende duende
https://statsroyale.com/es/dragon dragon
https://statsroyale.com/es/giant gigante


#### clases de recurso

In [38]:
recurso = URIRef(lista_diccionario[1]['link'])
for s in g.transitive_objects(recurso, RDF.type):
  print(g.value(s, name))

Gólem
gigante
tropas


#### Obtener la calidad de una carta

In [39]:
#obtener calidad
carta = URIRef(lista_diccionario[1]['link'])
for s in g.objects(carta,quality):
    print(g.value(s,quality))

None


#### Obtener todos los atributos de la carta

In [41]:
def atributos(card):
    atributos = list(lista_diccionario[1].keys())
    for s in g.transitive_subjects(None,card):
        for i in range(len(atributos)):
            if(g.value(s,atributos[i])!= 'None'):
                print(atributos[i],":",g.value(s,atributos[i]))
atributos(carta)

name : None
level : None
quality : None
objective : None
scope : None
speed : None
radio : None
timeoflife : None
lifepoints : None
hurt : None
damage : None
areadamage : None
link : None
classes : None


##### Obtener los daños de cada carta

In [43]:
!pip install tabulate

In [44]:
from tabulate import tabulate
#Daños
tipos = [n.troop,n.build,n.spell]
lista=[[]]
for tipo in tipos:
    for carta in g.subjects(RDF.type,tipo):
        if g.value(carta, damage)!=None:
            lista.append([g.value(carta, name),'Daño',g.value(carta, damage)])
        elif g.value(carta, areadamage)!=None:
            lista.append([g.value(carta, name),'Daño en Área',g.value(carta, areadamage)])
        elif g.value(carta, hurt)!=None:
            lista.append([g.value(carta, name),'Daño por segundo',g.value(carta, hurt)])
print(tabulate(lista,headers=['Nombre de la carta', 'Tipo de daño','Daño']))


Nombre de la carta      Tipo de daño      Daño
----------------------  ----------------  ------------------------------------------------------------
Gólem de hielo          Daño              28
Leñador                 Daño              250
Murciélagos             Daño              51
Chispitas               Daño              275
Esbirros                Daño              84
Globo bombástico        Daño              266
Gigante noble           Daño              149
Montacarneros           Daño              122
Dragón infernal         Daño              75–875
Pillos                  Daño              73
Arquero mágico          Daño              100
Mago de hielo           Daño              44
Espíritu de hielo       Daño en Área      91
Pandilla de duendes     Daño              90
Bruja                   Daño              100
Ariete de batalla       Daño por segundo  220
Mago                    Daño              167
Sabueso de lava         Daño              34
Bárbaros                Dañ

#### Clasificar por clase tropas, estructuras y hechizos

In [45]:
#Clasificar por clase tropas, estructuras y hechizos
tipos = [n.troop,n.build,n.spell]
for tipo in tipos:
    print("...............",(g.value(tipo, name)).upper(),"...............")
    for carta in g.subjects(RDF.type,tipo):
        print(g.value(carta, name))

............... TROPAS ...............
Gólem de hielo
Leñador
Murciélagos
Chispitas
Esbirros
Globo bombástico
Gigante noble
Montacarneros
Barril de esqueletos
Dragón infernal
Pillos
Arquero mágico
Mago de hielo
Espíritu de hielo
Pandilla de duendes
Bruja
Ariete de batalla
Mago
Sabueso de lava
Bárbaros
Duendes con lanza
Duende gigante
Gólem
Bebé dragón
Ejército de esqueletos
Bombardero
Lanzarrocas
Duende lanzadardos
Arqueras
Duendes
Esqueletos
Montapuercos
Gigante eléctrico
Cañón con ruedas
Guardias
Electrocutadores
Lanzafuegos
Rompemuros
Mago eléctrico
Reclutas reales
Dragón eléctrico
Gigante
Espíritus de fuego
Bárbaros de élite
Puercos reales
Bruja madre
Megaesbirro
Bruja nocturna
Mini P.E.K.K.A.
P.E.K.K.A.
Esqueleto gigante
Espíritu eléctrico
Verdugo
Horda de esbirros
Gólem de elixir
Dragones esqueleto
Fantasma real
............... ESTRUCTURAS ...............
Cañón
Choza de duendes
Torre Tesla
Jaula del forzudo
Ballesta
Lápida
Mortero
Torre infernal
Torre bombardera
Recolector de eli

##### Subir nivel a una carta

In [46]:
#Subir nivel a una carta
def agregarNivel(x):
    for s,d,o in g.triples((x,RDF.type,None)):
        nameX=g.value(s,name)
        nivelX=int(g.value(s,level))
        nivelX+=1
    return nivelX

In [47]:
#Prueba de subir nivel
agregarNivel(URIRef("https://statsroyale.com/es/card/Musketeer"))

10

#### Pelea entre dos cartas.

In [48]:
#funcion combate
def combat(x,lifeX,y,lifeY):
    for s,d,o in g.triples((x,RDF.type,None)):
        nameX=g.value(s,name)
        if g.value(s,damage)==None:
            damX=int(g.value(s,damage))
        elif g.value(s,areadamage)!=None:
            damX=int(g.value(s,areadamage))
        elif g.value(s,hurt)!=None:
            damX=int(g.value(s,hurt))
    for s,d,o in g.triples((y,RDF.type,None)):
        nameY=g.value(s,name)
        if g.value(s,damage)==None:
            damY=int(g.value(s,damage))
        elif g.value(s,areadamage)!=None:
            damY=int(g.value(s,areadamage))
        elif g.value(s,hurt)!=None:
            damY=int(g.value(s,hurt))
    lifeX-=damX
    lifeY-=damY
    if lifeX<=0:
        print(nameX,' vencido')
        return [0,lifeY]
    elif lifeY<=0:
        print(nameY,' vencido')
        return [lifeX,0]
    else:
        print('Ambos siguen luchando!')
        return [lifeX,lifeY]

In [49]:
#prueba de combate
musketer=URIRef("https://statsroyale.com/es/card/Musketeer")
GiantSkeleton=URIRef("https://statsroyale.com/es/card/Dark+Prince")

lifeMusketer,lifeGiantSkeleton=combat(musketer,400,GiantSkeleton,600)
print(lifeMusketer)
print(lifeGiantSkeleton)

Ambos siguen luchando!
219
394
